https://portal.inmet.gov.br/dadoshistoricos

In [1]:
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
import re

import warnings
warnings.filterwarnings('ignore')

In [3]:
# EXEMPLO DE UM ANO, para arquivos de Porto Alegre

path = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip'
r = requests.get(path, verify = False)
files = ZipFile(BytesIO(r.content))
pattern = re.compile("Porto Alegre", flags = re.IGNORECASE)
arquivo = []
for file in files.namelist():
    if re.search(pattern, str(file)):
        arquivo.append(file)
temp_poa = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", skiprows=8)

temp_poa = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", nrows=7, header = None)
temp_poa

In [2]:
# 2000 - 2021 TODOS ESTADOS
df = pd.DataFrame()
for ano in range(2000, 2022):
    print(ano)
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = []
    for file in files.namelist():
        if file[-4:].lower() == ".csv":
            arquivos.append(file)
        else:
            continue
    df01 = pd.DataFrame()
    for arquivo in arquivos:
        #print(arquivo)
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        estacao = info.iloc[2, 1]
        estado = info.iloc[1, 1]
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
        df02["ESTACAO"] = estacao
        df02["UF"] = estado
        df01 = pd.concat([df01, df02])
    df = pd.concat([df, df01])

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [3]:
# 2022
path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip'
r = requests.get(path, verify = False)
files = ZipFile(BytesIO(r.content))
arquivos = []
for file in files.namelist():
    if file[-4:].lower() == ".csv":
        arquivos.append(file)
    else:
        continue   
df_2022 = pd.DataFrame()
for arquivo in arquivos:
    info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
    estacao = info.iloc[2, 1]
    estado = info.iloc[1, 1]
    df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
    df02["ESTACAO"] = estacao
    df02["UF"] = estado
    df_2022 = pd.concat([df_2022, df02])

In [4]:
df_final = pd.concat([df, df_2022])

In [6]:
df_final.UF.unique()

array(['DF', 'BA', 'AM', 'RJ', 'RS', 'GO', 'MS', 'SP', 'MT', 'MG', 'AL',
       'CE', 'MA', 'PE', 'PI', 'RN', 'SE', 'PA', 'PR', 'SC', 'PB', 'TO',
       'ES', 'RO', 'AC', 'AP', 'RR'], dtype=object)

In [8]:
df_final.to_csv("temp_01012000_30062022.csv")